In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

DATA_PATH: Path = NOTEBOOK_PATH.parent.parent.joinpath('data')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

print(f'当前运行目录：{NOTEBOOK_PATH}')
print(f'当前数据目录：{DATA_PATH}')

当前运行目录：d:\Development\Jupyter\InvestmentNotebook\notebook\twinning_theory
当前数据目录：d:\Development\Jupyter\InvestmentNotebook\data


# 缠论尝试

## 数据

使用分钟数据。

In [2]:
symbol: str = 'SHFE.al2112'
period: str = 'Minute'

## 设置 pandas

In [3]:
import pandas as pd

pd.set_option('max_columns', None)
pd.set_option('display.width', 200)

## 加载数据

In [4]:
from pathlib import Path

def load_bar_data(data_file: Path, datetime_index: bool = True) -> pd.DataFrame:
    """
    加载 bar 数据为 pandas.DataFrame。
    :param data_file:
    :param datetime_index:
    :return:
    """
    if not data_file.exists():
        raise FileNotFoundError(f'{data_file} not found.')

    if datetime_index:
        return pd.read_csv(data_file, parse_dates=['datetime'], index_col=['datetime'])
    else:
        return pd.read_csv(data_file, parse_dates=['datetime'])

In [5]:
origin_data: pd.DataFrame = load_bar_data(DATA_PATH.joinpath(f'{symbol}_{period}.csv'))
df_ohlc: pd.DataFrame = origin_data.loc['2021-10-15 13:30:00':'2021-10-19 00:59:00', ['open', 'high', 'low', 'close']]

df_ohlc.info()
print('-' * 100)
df_ohlc.head(5)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 795 entries, 2021-10-15 13:30:00 to 2021-10-19 00:59:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    795 non-null    float64
 1   high    795 non-null    float64
 2   low     795 non-null    float64
 3   close   795 non-null    float64
dtypes: float64(4)
memory usage: 31.1 KB
----------------------------------------------------------------------------------------------------


,open,high,low,close
datetime,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0


In [6]:
# df_tt: pd.DataFrame = pd.DataFrame(columns=['high_merged', 'low_merged', '区间高点', '区间低点', '区间突破', '包含', '笔'], index=[i for i in range(df_ohlc.shape[0])])
df_tt: pd.DataFrame = pd.DataFrame(columns=['是否包含', '包含方向','前K方向', '合并后高点', '合并后低点', '合并后周期', '区间高点', '区间低点', '区间突破', '笔'], index=df_ohlc.index)

df_tt.head(5)

,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,
2021-10-15 13:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:31:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 模拟处理

第一根K线，笔肯定无效。

第二根开始区分情况：

1. 无包含关系：
    1. 第二根的高点大于第一根的高点：笔的起始位置为第一根K线的低点，目前画向第二根K线的高点，为上升笔。
    2. 第二根的低点小于第一根的低点：笔的起始位置为第一根K线的高点，目前画向第二根K线的低点，为下降笔。
2. 有包含关系：
    不能确定。
    
**问题：第二根要不要合并？**

**回答：** 处理，但是因为不知道处理方向，就去最高点与最低点合并为一根K线。

In [7]:
def merge_candlestick(idx: int):
    """
    处理K线合并。
    """
    # ----------------------------------------
    # 第1根K线需要特殊处理。
    # ----------------------------------------
    if idx == 0:
        df_tt.iloc[idx].at['是否包含'] = None
        df_tt.iloc[idx].at['包含方向'] = None
        df_tt.iloc[idx].at['前K方向'] = None
        df_tt.iloc[idx].at['合并后高点'] = df_ohlc.iloc[idx].at['high']
        df_tt.iloc[idx].at['合并后低点'] = df_ohlc.iloc[idx].at['low']
        df_tt.iloc[idx].at['合并后周期'] = 1
        df_tt.iloc[idx].at['区间高点'] = df_ohlc.iloc[idx].at['high']
        df_tt.iloc[idx].at['区间低点'] = df_ohlc.iloc[idx].at['low']
        df_tt.iloc[idx].at['区间突破'] = None
        df_tt.iloc[idx].at['笔'] = None
    else:
        # ----------------------------------------
        # 处理：包含。
        # ----------------------------------------
        
        # 后K包含前K。
        if df_ohlc.iloc[idx].at['high'] >= df_ohlc.iloc[idx-1].at['high'] and df_ohlc.iloc[idx].at['low'] <= df_ohlc.iloc[idx-1].at['low']:
            df_tt.iloc[idx].at['是否包含'] = True
            df_tt.iloc[idx].at['包含方向'] = '向前'
            
        # 前K包含后K。
        elif df_ohlc.iloc[idx].at['high'] <= df_ohlc.iloc[idx-1].at['high'] and df_ohlc.iloc[idx].at['low'] >= df_ohlc.iloc[idx-1].at['low']:
            df_tt.iloc[idx].at['是否包含'] = True
            df_tt.iloc[idx].at['包含方向'] = '向后'

        # 不包含
        else:
            df_tt.iloc[idx].at['是否包含'] = False
            df_tt.iloc[idx].at['包含方向'] = False
            
            df_tt.iloc[idx].at['区间突破'] = False
        
        
        # ----------------------------------------
        # 处理：前K方向。
        # 前K方向有：
        # None（第1、2根K线）、包含、向上、向下
        # ----------------------------------------
        
        # 第2根K线，不存在前K方向。
        if idx == 1:
            df_tt.iloc[idx].at['前K方向'] = None
        else:
            if df_tt.iloc[idx].at['是否包含']:
                df_tt.iloc[idx].at['前K方向'] = '包含'
            else:
                # 前1根K线高点
                k1h = df_tt.iloc[idx-1].at['合并后高点']
                # 前1根K线低点
                k1l = df_tt.iloc[idx-1].at['合并后低点']
                # 前2根K线距离
                d = df_tt.iloc[idx-1].at['合并后周期']
                
                if d == idx:
                    # 从第1根K线起，到前1根K线的所有K线都被合并。
                    # 只需比较当前K线和之前1根。
                    df_tt.iloc[idx].at['前K方向'] = '包含'
                else:    
                    # 前2根K线高点
                    k2h = df_tt.iloc[idx-d].at['合并后高点']
                    # 前2根K线低点
                    k2l = df_tt.iloc[idx-d].at['合并后低点']

                    if k1h > k2h and k1l > k2l:
                        df_tt.iloc[idx].at['前K方向'] = '向上'
                    elif k1h < k2h and k1l < k2l:
                        df_tt.iloc[idx].at['前K方向'] = '向下'
                    else:
                        df_tt.iloc[idx].at['前K方向'] = '【出错啦】'
                        print('【出错啦】')
                        print(f'd = {d}')
                        print(f'k1h = {k1h}')
                        print(f'k1l = {k1l}')
                        print(f'k2h = {k2h}')
                        print(f'k2l = {k2l}')

        # ----------------------------------------
        # 处理：合并。
        # ----------------------------------------
        if not df_tt.iloc[idx].at['前K方向'] or df_tt.iloc[idx].at['前K方向'] == '包含':
            df_tt.iloc[idx].at['合并后高点'] = max(df_tt.iloc[idx-1].at['合并后高点'], df_ohlc.iloc[idx].at['high'])
            df_tt.iloc[idx].at['合并后低点'] = min(df_tt.iloc[idx-1].at['合并后低点'], df_ohlc.iloc[idx].at['low'])
            df_tt.iloc[idx].at['合并后周期'] = df_tt.iloc[idx-1].at['合并后周期'] + 1
        
        # 区间高低点。
        if df_ohlc.iloc[idx].at['high'] > df_tt.iloc[idx-1].at['区间高点']:
            df_tt.iloc[idx].at['区间突破'] = '向上'
            df_tt.iloc[idx].at['区间高点'] = df_ohlc.iloc[idx].at['high']
            df_tt.iloc[idx].at['区间低点'] = df_tt.iloc[idx-1].at['区间低点']

        elif df_ohlc.iloc[idx].at['low'] < df_tt.iloc[idx-1].at['区间低点']:
            df_tt.iloc[idx].at['区间突破'] = '向下'
            df_tt.iloc[idx].at['区间低点'] = df_ohlc.iloc[idx].at['low']
            df_tt.iloc[idx].at['区间高点'] = df_tt.iloc[idx-1].at['区间高点']

        elif df_ohlc.iloc[idx].at['high'] > df_tt.iloc[idx-1].at['区间高点'] and df_ohlc.iloc[idx].at['low'] < df_tt.iloc[idx-1].at['区间低点']:
            df_tt.iloc[idx].at['区间突破'] = '双向'
            df_tt.iloc[idx].at['区间高点'] = df_ohlc.iloc[idx].at['high']
            df_tt.iloc[idx].at['区间低点'] = df_ohlc.iloc[idx].at['low']

        else:
            df_tt.iloc[idx].at['区间突破'] = False
            df_tt.iloc[idx].at['区间高点'] = df_tt.iloc[idx-1].at['区间高点']
            df_tt.iloc[idx].at['区间低点'] = df_tt.iloc[idx-1].at['区间低点']

        df_tt.iloc[idx].at['笔'] = None

### 第1根

In [8]:
idx: int = 0
merge_candlestick(idx)

rows_to_watch: int = 5
result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第2根

In [9]:
idx = 1
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第3根

In [10]:
idx = 2
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第4根

In [11]:
idx = 3
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第5根

In [12]:
idx = 4
merge_candlestick(idx)

rows_to_watch = 10
result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第6根

In [13]:
idx = 5
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,True,向前,包含,23900.0,23830.0,6,23900.0,23830.0,False,None
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第7根

In [14]:
idx = 6
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,True,向前,包含,23900.0,23830.0,6,23900.0,23830.0,False,None
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,False,False,包含,23900.0,23830.0,7,23900.0,23830.0,False,None
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第8根

In [15]:
idx = 7
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,True,向前,包含,23900.0,23830.0,6,23900.0,23830.0,False,None
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,False,False,包含,23900.0,23830.0,7,23900.0,23830.0,False,None
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,False,False,包含,23915.0,23830.0,8,23915.0,23830.0,向上,None
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 第9根

In [16]:
idx = 8
merge_candlestick(idx)

rows_to_watch = 15
result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,True,向前,包含,23900.0,23830.0,6,23900.0,23830.0,False,None
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,False,False,包含,23900.0,23830.0,7,23900.0,23830.0,False,None
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,False,False,包含,23915.0,23830.0,8,23915.0,23830.0,向上,None
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,False,False,包含,23915.0,23830.0,9,23915.0,23830.0,False,None


### 第10根

In [17]:
idx = 9
merge_candlestick(idx)

result = pd.concat([df_ohlc.iloc[:rows_to_watch], df_tt.iloc[:rows_to_watch]], axis=1)
result

,open,high,low,close,是否包含,包含方向,前K方向,合并后高点,合并后低点,合并后周期,区间高点,区间低点,区间突破,笔
datetime,,,,,,,,,,,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0,None,None,None,23900.0,23840.0,1,23900.0,23840.0,None,None
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0,True,向后,None,23900.0,23840.0,2,23900.0,23840.0,False,None
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0,True,向后,包含,23900.0,23840.0,3,23900.0,23840.0,False,None
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0,True,向前,包含,23900.0,23840.0,4,23900.0,23840.0,False,None
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0,False,False,包含,23900.0,23830.0,5,23900.0,23830.0,向下,None
2021-10-15 13:35:00,23830.0,23880.0,23830.0,23880.0,True,向前,包含,23900.0,23830.0,6,23900.0,23830.0,False,None
2021-10-15 13:36:00,23880.0,23900.0,23875.0,23895.0,False,False,包含,23900.0,23830.0,7,23900.0,23830.0,False,None
2021-10-15 13:37:00,23895.0,23915.0,23880.0,23885.0,False,False,包含,23915.0,23830.0,8,23915.0,23830.0,向上,None
2021-10-15 13:38:00,23885.0,23890.0,23845.0,23860.0,False,False,包含,23915.0,23830.0,9,23915.0,23830.0,False,None
